# Gradient Boosting Decision Trees

GBDT is a boosting algorithm, which is a type of an ensemble learning method that operates in a sequential manner. GBDT employs a training data set to train a set of decision trees. The algorithm attempts to identify an instance within the dataset that corresponds to a leaf node, and then aggregates the leaf predictions of all trees to arrive at a final instance prediction. The objective of the algorithm is to minimize the loss function by making use of the predictions made by the preceding tree. This implies that each decision tree is fitted to the residuals from the preceding tree. 
In order to evaluate the performance of a decision tree, the gain is employed in order to identify the optimal splitting points in all leaf nodes. This is achieved by computing the gain and then attempting to ascertain the most optimal splitting point by using of the highest information gain. Once a new splitting point has been established, the current tree is complete, and the algorithm proceeds sequentially with the next tree.

![image.png](images/figure_3.png)

Figure 3: The way of functioning of a GBDT

As stated by Fu et al. (2019), the majority of the computing power is required for the identification of the splitting points within the tree structure. Two methods are employed for identifying the splitting points: the pre-sorted algorithm and the histogram-based algorithm. Firstly, an overview of both algorithms will be presented, after which a more detailed examination of the histogram-based algorithm will be undertaken. 

<br>**Pre-Sorted Algorithm**: As explained by Ke et al. (2017), this algorithm identifies the optimal split points by enumerating all potential split points on pre-sorted feature values. This straightforward algorithm is capable of identifying the optimal split points. The process of determining the optimal split point requires a significant amount of storage and computational resources.
<br>**Histogram-Based Algorithm**: As demonstrated by Fu et al. (2019), this algorithm employs a discrete approach to categorize the features within the dataset, assigning each feature a specific integer value. The bins are utilized for the construction of feature histograms: A histogram displays numerical data by grouping the data into "bins" of equal width. Each bin can be plotted as a bar whose height corresponds to how many data points are in that bin. The feature histograms include only those features that have reached the specified node. The aforementioned histograms are employed in order to identify the optimal splitting points from the feature.<br>

As previously stated by Ke et al. (2017), the algorithmic processing time increases with an increasing number of data points. However, the histogram-based algorithm demonstrates superior efficiency in terms of processing speed and memory consumption. The objective of LightGBM is to enhance GBDT for handling large datasets, which is why it is based on the histogram-based algorithm. <br>

The **Histogram-Based Algorithm** algorithm is widely used because it identifies precise splitting points with minimal computational expense and storage, solely by considering a fixed number of splitting points for a given feature, eliminating the need to search across an entire continuous range. The most common approach to this is the use of the quantile sketch to approximate the feature distribution. To this end, all instances from a given tree node are enumerated and their first- and second-order gradients are accumulated to create two feature histograms. The optimal local splitting point can be identified by calculating the gain of the splitting points. The optimal global splitting point is defined as the point with the highest gain over all features.

To enhance the algorithm's efficiency and memory utilization, LightGBM employs alternative techniques, such as GOSS (Gradient One-Side Sampling) and EFB (Exclusive Feature Bundling). The objective of GOSS is to reduce the number of data instances in order to minimize memory consumption and reduce the computational cost. EFB is employed for the purpose of reducing the number of features. The following two chapters will provide a further explanation of both. 

## Practical Understanding of the Histogram-Based Algorithm 

In this section we aim to explain how the histogram-based split finding works. We only consider the data in one specific node of an imaginary decision tree of a GBDT in this practical section. The same process is applied in all internal nodes of each decision tree in a GBDT. 

The figure below shows a detailed explanation of the algorithm. The algorithm gets three inputs. The first is the training data $I$, the input $d$ defines the maximal depth to prevent overfitting. $m$ describes the feature dimension. The algorithm first initialises the nodeSet and rowSet. This can be understood as a table, the nodeSet starts with 0 and defines the first node, the rowSet contains the indices of data in this node. After splitting, the nodeSet contains a second level and every node in this level receives its own set of indices. The algorithm is a big for-loop which does only iterations to the maximum depth of a tree. The outer for-loop consists of smaller loops. In the first, the algorithm iterates over the nodes, it loads the rowSet for every node and builds a histogram for every feature from this data. The bins get the values from a function defined as *I.y[j]*. This means that the accumulated residuals $g$ get calculated by this formula: 
$$g = -(y-\hat{y})$$
The line $H[bin].n -> H[bin].n + 1$ can be represented by two formulas. In context of regression, the Mean Square Error (MSE) is used and in context of classification, the binary cross entropy is used which is calculated by 
$$h = \hat{y}*(1-\hat{y}) $$
Finally, the algorithm searches the best split point in the feature-histograms and updates the nodeSet and rowSet according to the best spliting points. In the next sections, we take a closer look at finding the best splitting points.

![image.png](images/figure_7.PNG)

Figure 4: Explanation of histogram-based split finding

To show how histogram-based split finding works in detail, we are using numpy, pandas and plotly.express. As a data set, we import the iris dataset from plotly and prepare it to fit in the context of binary classification. We only use two flower types from the original three: Virginica Setosa and Versicolor are chosen. In this way, we change the objective to binary classification.

In [1]:
# Run helper code in utilities
%run ../utilities.ipynb

In [2]:
# Create simplified DataFrame
df = px.data.iris()
df = df[df["species_id"] <= 2]
df.loc[df["species_id"] == 2, "species_id"] = 0
df.loc[df["species_id"] == 1, "species_id"] = 1

In [3]:
# Display DataFrame
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1


In the figure below, you can see the dataset visualized as a scatterplot of sepal length and width of the flowers. 

In [4]:
# Define, style and display scatterplot
fig_data = px.scatter(df, x="sepal_length", y="sepal_width", color="species")
style_scatterplot(fig_data)
fig_data.show()

Now we implement the split finding algorithm as a function. *histogram-based-split_finding* constructs a histogram for each feature, stores it in a pandas.DataFrame and returns it with three columns: *split* points, *sum* and *score*. The *split* represents the possible split points, the *sum* represents the sum of the residuals and the *score* is derived from the sum and split and represents the value of a split. The function takes the data set, an initial *prediction*, a *label* as the the class an instance belongs to and a list of *features* as arguments. The *prediction* refers to a relative value as a prediction of class belonging (e.g. 0.4 assumes that an instance rather belongs to class 0). The algorithm does not yet solve the split finding problem but it returns a DataFrame with which we can easily solve it in a next step. This whole explanation aims to solve the problem visually for a better understanding of future steps. 

In [5]:
def histogram_based_split_finding(
    I: pd.DataFrame, prediction: float=0.5, label:str ="label", features: list=["x", "y"]
) -> dict:
    """A simplified implementation of a histogram based split finding algorithm

    Args:
        I: the data set used for training
        prediction: relative value as a prediction of class belonging
        label: the label of the class instances belong to
        features (list): a list of the features of the train data

    Returns:
        results (dict): a dictionary containing a histogram per feature
    """
    # Calculate residuals as difference between binary label and prediction
    # This represents the error
    I["gradients"] = I[label] - prediction
    h = prediction * (1 - prediction)

    # Calculate score for split point
    def score(split_list, length):
        return np.power(np.sum(split_list), 2) / length

    results = {}

    for feature in features:
        # Construct histogram per feature
        g_sum, bins = np.histogram(I[feature], bins=9, weights=I["gradients"])

        scores = []
        for i in range(len(bins) - 1):
            # Compute the left and right side of possible histogram splits
            left_length = len(I[I[feature] <= bins[i]])
            length_right = len(I[I[feature] > bins[i]])

            # Compute score for left and right side
            score_left = score(g_sum[:i], left_length)
            score_right = score(g_sum[i:], length_right)

            # Compute score for every split
            total_score = score_left + score_right
            scores.append(total_score)

        # Construct the Dataframe
        scores.append(0)  # Make sure the scores fit the Dataframe
        results[feature] = pd.DataFrame(
            {"split": bins, "sum": np.append(g_sum, 0), "score": scores}
        )

    return results

Now we run the algorithm with the iris data. 0.5 is chosen as a placebo prediction. As we only condider a part of a decision tree here, we just assume this value. The label is species_id, which is binary in our context. For education purposes, we only take sepal_length and sepal_width into consideration. 

In [6]:
r = histogram_based_split_finding(
    I=df, prediction=0.5, label="species_id", features=["sepal_length", "sepal_width"]
)

With the following code snippet, we calculate the best local splitting point. In this part of the code, we also visualize the data. As outputs, we get two histograms. The first histogram represents the score and the second histogram represents the sum of residuals of every bin. In addition, the algorithm looks for the maximum score and draws this threshold as a red line in the figure. This red line represents the best split point.

In [7]:
def create_histogram(data: pd.DataFrame, features: list=["split", "score", "sum"]) -> list[go.Figure]:
    """Helper function which returns two histograms for sum and score for a given dataset. It calculates and displays the best possible split point.

    Args:
        data (pd.DataFrame): the date the histograms is to be plotted for
        features (list): a list containing the features of the data set

    Returns:
        figures (list[go.Figure]): a list of two histograms
    """
    # Initialize figures
    figures = [
        px.bar(data, x=features[0], y=features[1], title=features[1]),
        px.bar(data, x=features[0], y=features[2], title=features[2]),
    ]
    
    # Calculate best split point
    best_split = data.loc[data["score"].idxmax(), "split"]
    
    tab = (
        [
            data.loc[data["score"].idxmax(), "score"],
            data.loc[data["score"].idxmin(), "score"],
        ],
        [data.loc[data["sum"].idxmax(), "sum"], data.loc[data["sum"].idxmin(), "sum"]],
    )
    
    # Create figures
    for figure in zip(figures, tab):
        figure[0].add_shape(
            type="line",
            x0=best_split,
            x1=best_split,  # Position of the line on the x axis
            y0=figure[1][1],
            y1=figure[1][0],  # Position of the line on the y axis
            line=dict(color="red", width=2),
        )
        
    return figures

Now we display the histograms of the sepal_length feature. In the figure *sum*, you can see the histogram of errors. You can see that the left side is negative valued and the right positive valued. The change from negative to positive represents the best splitting point. In the second figure wich represents the score, you can see that the red line has its position at at the maximum score. 

In [8]:
# Print histograms for sepal_length
histograms_sepal_length = create_histogram(
    data=r["sepal_length"], features=["split", "score", "sum"]
)

# Style and display histograms
style_bar_chart(histograms_sepal_length[1])
histograms_sepal_length[1].show()
style_bar_chart(histograms_sepal_length[0])
histograms_sepal_length[0].show()

Now we calculate the local best splitpoint for the feature sepal_width. In these figures you can see the same kind of distributions and the split point is also on the maximum of the score, but now the question is what the global best split point is. We have two features and two splitpoints. The global best splitting point is defined by the maximum score of all the features. The feature with the highest score is used to split the data on this node in the end.

In [9]:
# Print histograms for sepal_width
histograms_sepal_width = create_histogram(
    data=r["sepal_width"], features=["split", "score", "sum"]
)

# Style and display histograms
style_bar_chart(histograms_sepal_width[1])
histograms_sepal_width[1].show()
style_bar_chart(histograms_sepal_width[0])
histograms_sepal_width[0].show()

In our particular case, the maximum score of sepal_length is 5.50 and sepal_width is 3.06. The score of sepal_length is larger than the maximum score of sepal_width. This means that the tree is split at this particular node at feature sepal_length at a value of 5.50. In the figure below, both lines are plotted, the red one representing the global best split point at the sepal_length feature and the green one representing the best split point for sepal_width. The red line thus shows the final split point. The instance space is separated at the red line into two subsets. Those subsets both contain a more uniform population than the unsplitted set.

In [10]:
fig_final = px.scatter(df, x="sepal_length", y="sepal_width", color="species")

fig_final.add_shape(
    type="line",
    x0=5.5,
    x1=5.5,  # Position of the lines on the x axis
    y0=1.5,
    y1=4.7,  # Position of the lines on the y axis
    line=dict(color="red", width=2),
)
fig_final.add_shape(
    type="line",
    x0=4,
    x1=8,  # Position of the lines on the x axis
    y0=3,
    y1=3,  # Position of the lines on the y axis
    line=dict(color="green", width=2),
)

# Style and display scatterplot
style_scatterplot(fig_final)
fig_final.show()

The presented steps are repeated in all other internal nodes of the GBDT. Both retrieved subsets are subject to the same procedure, where again each subset is splitted into two subsets using histogram based split finding.

[<<<](1.2_Intro_to_LightGBM.ipynb)  | 1.3_gradient_boosting_decision_trees |  [>>>](1.4_gradient_based_one_side_sampling.ipynb)